### 토큰화

In [2]:
# !pip install nltk 처음한번

import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
import pandas as pd

In [5]:
df = pd.read_csv('preprocessed/preprocessed_done.csv')

#잘 생성됐는지 확인
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Country Code     1574 non-null   object
 1   School Name      1574 non-null   object
 2   Review Link      1574 non-null   object
 3   Review Title     1574 non-null   object
 4   Review Content   1574 non-null   object
 5   cleaned_content  1574 non-null   object
 6   preprocessed     1574 non-null   object
dtypes: object(7)
memory usage: 86.2+ KB


,Country Code,School Name,Review Link,Review Title,Review Content,cleaned_content,preprocessed
0,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,"소중한 경험, 잊지 못할 기억, 쿠프슈타인",다만 집주인이 관리를 잘 안 해주고 규칙도 명확하지 않아서 청소나 화장실 이용 등...,다만 집주인이 관리를 잘 안 해주고 규칙도 명확하지 않아서 청소나 화장실 이용 등...,"[['집주인', '관리', '안', '규칙', '청소', '화장실', '이용', '..."
1,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,언제나 마음속의 고향인 쿠프슈타인,"비슷한 가격대에 비교적 더 자유롭게 외부에서 자취도 할 수 있지만, 기숙사만의 좋...",비슷한 가격대에 비교적 더 자유롭게 외부에서 자취도 할 수 있지만 기숙사만의 좋...,"[['가격', '비교', '더', '외부', '자취', '기숙사', '점', '사전..."
2,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,쿠프슈타인에서의 한 학기,"사람 사는곳 다 똑같고, 생각한대로 다 되지 않는다는 이야기는 익히 듣고 와서 그...",사람 사는곳 다 똑같고 생각한대로 다 되지 않는다는 이야기는 익히 듣고 와서 그...,"[['사람', '곳', '생각', '대로', '이야기', '그게', '크게', '스..."
3,AT,FH Kufstein Tirol University of Applied Sciences,https://oia.yonsei.ac.kr/partner/expReport.asp...,여유롭고 소중했던 쿠프슈타인에서의 시간,불안해서 국제처에도 계속 문의하고 쿠프슈타인 대학교에도 계속 문의했었는데 계속 기...,불안해서 국제처에도 계속 문의하고 쿠프슈타인 대학교에도 계속 문의했었는데 계속 기...,"[['국제', '처', '계속', '문의', '쿠프슈타인', '대학교', '계속',..."
4,AT,Johannes Kepler Universitat Linz,https://oia.yonsei.ac.kr/partner/expReport.asp...,"유럽의 심장, 평화롭고 선명했던 린츠를 그리워하며",흐린 날씨가 지속되면 처음에는 조금 우울하지만 그래도 이것이 유럽의 겨울이구나 하...,흐린 날씨가 지속되면 처음에는 조금 우울하지만 그래도 이것이 유럽의 겨울이구나 하...,"[['날씨', '지속', '처음', '유럽', '겨울'], ['오스트리아', '문화..."


In [11]:
# Assuming df['cleaned_content'] contains the preprocessed reviews
tokenizer = RegexpTokenizer(r'\w+')

# Tokenize each review and convert to lowercase
df['tokens'] = df['preprocessed'].apply(lambda x: tokenizer.tokenize(x.lower()))

df['tokens']

0                 [집주인, 관리, 안, 규칙, 청소, 화장실, 이용, 문제, 스트레스]
1       [가격, 비교, 더, 외부, 자취, 기숙사, 점, 사전, 살, 곳, 실제, 보지, ...
2                   [사람, 곳, 생각, 대로, 이야기, 그게, 크게, 스트레스, 요]
3       [국제, 처, 계속, 문의, 쿠프슈타인, 대학교, 계속, 문의, 계속, 연락, 나중...
4       [날씨, 지속, 처음, 유럽, 겨울, 오스트리아, 문화, 충격, 충격, 오스트리아,...
                              ...                        
1569              [때문, 영어, 렁증, 발표, 공포증, 가지, 전형, 처음, 스트레스]
1570                                             [템포, 박자]
1571                   [사람, 관계, 지속, 때문, 적극, 성격, 스트레스, 수도]
1572    [달, 정도, 스트레스, 좌절, 어회, 책, 전자사전, 항상, 붙이, 점점, 적응,...
1573    [처음, 언어, 뿐, 환경, 변화, 스트레스, 정말, 처음, 시작, 수영, 수업, ...
Name: tokens, Length: 1574, dtype: object

### 토픽 모델링

In [7]:
!pip install gensim

from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


                                              0.0/24.0 MB ? eta -:--:--
                                              0.2/24.0 MB 3.3 MB/s eta 0:00:08
                                              0.3/24.0 MB 3.5 MB/s eta 0:00:07
                                              0.5/24.0 MB 3.9 MB/s eta 0:00:07
     -                                        0.7/24.0 MB 3.5 MB/s eta 0:00:07
     -                                        0.8/24.0 MB 3.8 MB/s eta 0:00:07
     -                                        1.0/24.0 MB 3.7 MB/s eta 0:00:07
     -                                        1.2/24.0 MB 3.7 MB/s eta 0:00:07
     --                                       1.3/24.0 MB 3.7 MB/s eta 0:00:07
     --                                       1.5/24.0 MB 3.6 MB/s eta 0:00:07
     --                                       1.7/24.0 MB 3.7 MB/s eta 0:00:07
     ---                                      1.8/24.0 MB 3.6 MB/s eta 0:00:07
     ---                                      2.0/24.0 MB 3


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# 사전 생성 및 코퍼스 준비
# 사전 생성
dictionary = corpora.Dictionary(df['tokens'])

# 코퍼스 생성
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

2024-05-16 23:23:25,856 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-05-16 23:23:25,888 : INFO : built Dictionary<3269 unique tokens: ['관리', '규칙', '문제', '스트레스', '안']...> from 1574 documents (total 20160 corpus positions)
2024-05-16 23:23:25,889 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<3269 unique tokens: ['관리', '규칙', '문제', '스트레스', '안']...> from 1574 documents (total 20160 corpus positions)", 'datetime': '2024-05-16T23:23:25.889101', 'gensim': '4.3.2', 'python': '3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22631-SP0', 'event': 'created'}


In [13]:
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2024-05-16 23:23:27,461 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2024-05-16 23:23:27,462 : INFO : using serial LDA version on this node
2024-05-16 23:23:27,465 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 1574 documents, updating model once every 1574 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2024-05-16 23:23:27,466 : INFO : PROGRESS: pass 0, at document #1574/1574
2024-05-16 23:23:28,227 : INFO : optimized alpha [0.109022915, 0.12246187, 0.11270816, 0.12428228, 0.16868491]
2024-05-16 23:23:28,230 : INFO : topic #0 (0.109): 0.046*"스트레스" + 0.027*"수업" + 0.015*"때문" + 0.012*"생각" + 0.011*"곳" + 0.011*"정도" + 0.010*"과제" + 0.009*"경우" + 0.009*"점" + 0.007*"날씨"
2024-05-16 23:23:28,231 : INFO : topic #1 (0.122): 0.053*"스트레스" + 0.026*"때문" + 0.018*"수업" + 0.011*"학생" + 0.010*"곳" + 0.010*"말" + 0.009*"수도" + 0.009*"우울증" + 0.009*"사람" + 0.008*"공부"
2024-05-16 

In [16]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2024-05-16 23:24:01,895 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -2.6086.
[([(0.05207814, '스트레스'),
   (0.026136557, '때문'),
   (0.017046573, '수업'),
   (0.012497097, '우울증'),
   (0.011365523, '학생'),
   (0.010292296, '곳'),
   (0.009921294, '수도'),
   (0.009595216, '말'),
   (0.008879131, '공부'),
   (0.008519662, '사람'),
   (0.008027914, '경우'),
   (0.00744121, '시험'),
   (0.006322933, '정도'),
   (0.0057202806, '영어'),
   (0.0056814332, '학교'),
   (0.0056474227, '겨울'),
   (0.0056360387, '정말'),
   (0.0054008607, '날씨'),
   (0.005397638, '생각'),
   (0.0051312507, '처음')],
  -2.336891086066559),
 ([(0.070768364, '스트레스'),
   (0.02354834, '수업'),
   (0.017999444, '때문'),
   (0.014992525, '처음'),
   (0.014038874, '날씨'),
   (0.013768074, '정말'),
   (0.01177578, '생각'),
   (0.011063178, '영어'),
   (0.009313752, '교환학생'),
   (0.0073605203, '한국'),
   (0.0071148896, '친구'),
   (0.006868299, '사람'),
   (0.006797918, '학생'),
   (0.006517069, '경우'),
   (0.006000607, '해'),
   (0.0059503336, '집'),
   (0.005869076, '겨울'),
   (0.005834269, '매우'),
   (0.0058193724, '생활'

### 결과 시각화

In [19]:
!pip install pyLDAvis

import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/2.6 MB ? eta -:--:--
     --                                       0.1/2.6 MB 4.3 MB/s eta 0:00:01
     ----                                     0.3/2.6 MB 3.8 MB/s eta 0:00:01
     -------                                  0.5/2.6 MB 3.9 MB/s eta 0:00:01
     ----------                               0.7/2.6 MB 3.8 MB/s eta 0:00:01
     ----------                               0.7/2.6 MB 3.6 MB/s eta 0:00:01
     ---------------                          1.0/2.6 MB 3.7 MB/s eta 0:00:01
     -----------------                        1.1/2.6 MB 3.8 MB/s eta 0:00:01
     --------------------                     1.3/2.6 MB 3.7 MB/s eta 0:00:01
     ----------------------                   1.5/2.6 MB 3.6 MB/s eta 0:00:01
     -------------------------                1.6/2.6 MB 3.6 MB/s eta 0:00:01
     ----------------------------             1.8/2.6 MB 3.6 MB/s eta 0:00:01
     ------------------------------           2.0/2.6 MB 3.6 MB/s eta 0

In [22]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'display_topics.html')